In [2]:
# Full dataset 41000 images
# FILES_DIR = "s3://waleed-movies"

#small local testing dataset, 40,000 images
FILES_DIR = "/home/ec2-user/SageMaker/movie-posters"

NUM_WORKERS = 4 

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import json

import ray
from ray.train.torch import TorchPredictor, TorchCheckpoint
from ray.train.batch_predictor import BatchPredictor
from ray.data.preprocessors import BatchMapper
from ray.data.datasource import ImageFolderDatasource
import anyscale

from torchvision.models.detection.ssd import ssd300_vgg16

from util import visualize_objects, convert_to_tensor, SSDPredictor

# TODO: Enable auto casting once we resolve call_model() output format
from ray.data.context import DatasetContext
ctx = DatasetContext.get_current();
ctx.enable_tensor_extension_casting = False

In [3]:
def batch_predict(files_dir):
    dataset = ray.data.read_datasource(
        ImageFolderDatasource(), root=files_dir, size=(300, 300), mode="RGB"
    )
    preprocessor = BatchMapper(convert_to_tensor)
    model = ssd300_vgg16(pretrained=True)
    ckpt = TorchCheckpoint.from_model(model=model, preprocessor=preprocessor)
    predictor = BatchPredictor.from_checkpoint(ckpt, SSDPredictor)
    return predictor.predict(dataset, 
                             batch_size=128,
                             min_scoring_workers=NUM_WORKERS,
                             max_scoring_workers=NUM_WORKERS, 
                             num_cpus_per_worker=4, 
                             num_gpus_per_worker=1, 
                             feature_columns=["image"], 
                             keep_columns=["image"])





In [4]:
ray.init()

Usage stats collection is enabled by default for nightly wheels. To disable this, run the following command: `ray disable-usage-stats` before starting Ray. See https://docs.ray.io/en/master/cluster/usage-stats.html for more details.


2022-08-11 04:44:46,010	INFO worker.py:1490 -- Started a local Ray instance.


Python version:,3.8.12
Ray version:,3.0.0.dev0


In [ ]:
prediction_results = batch_predict(FILES_DIR)


Downloading: "https://download.pytorch.org/models/ssd300_vgg16_coco-b556d3b4.pth" to /home/ec2-user/.cache/torch/hub/checkpoints/ssd300_vgg16_coco-b556d3b4.pth


  0%|          | 0.00/136M [00:00<?, ?B/s]

Map Progress (4 actors 0 pending):   0%|          | 0/200 [00:08<?, ?it/s](BlockWorker pid=16460) /home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272172048/work/aten/src/ATen/native/TensorShape.cpp:2157.)
(BlockWorker pid=16460)   return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
(BlockWorker pid=16457) /home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272172048/work/aten/src/ATen/native/TensorShape.cpp:2157.)
(BlockWorker pid=16457)   return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
(BlockWorker pid=16459) /home/ec2-user/anaconda3/envs/